Issue [19](https://github.com/drivendataorg/cyanobacteria-prediction/issues/19)

Check that our planetary computer results are correct

In [1]:
import json

from cloudpathlib import AnyPath
import pandas as pd
from tqdm import tqdm

from cyano.config import FeaturesConfig
from cyano.data.utils import add_unique_identifier
from cyano.data.satellite_data import (
    select_items,
    search_planetary_computer,
    get_items_metadata
)

### Load data

In [2]:
test = pd.read_csv(
    AnyPath(
        "s3://drivendata-competition-nasa-cyanobacteria/experiments/splits/competition/test.csv"
    )
)
test = add_unique_identifier(test)
test.shape

(6510, 10)

In [3]:
test.head(2)

,uid,data_provider,region,latitude,longitude,date,density_cells_per_ml,severity,distance_to_water_m,log_density
sample_id,,,,,,,,,,
4a89ead93e2caa84da636236bb361e12,aabn,California Environmental Data Exchange Network,west,36.5597,-121.510,2016-08-31,5867500.0,4,3512.0,15.584939
a7e2d76f204ac347ae5529557eb7f665,aair,California Environmental Data Exchange Network,west,33.0426,-117.076,2014-11-01,2769000.0,4,195.0,14.833997


In [4]:
# Take a subset to run PC search on
samples = test.sample(n=100, random_state=3) #(n=100, random_state=3)
samples.region.value_counts()

region
west         36
midwest      26
south        24
northeast    14
Name: count, dtype: int64

### Define functions

In [5]:
# use the same geographic window as our original full PC search
config = FeaturesConfig(pc_meters_search_window=10000)
config.pc_meters_search_window, config.pc_days_search_window

(10000, 15)

Note that our original PC search includes a time buffer both before *and* after the sample, while our current package code only adds a time buffer *before* the sample. 

Rather than comparing the raw results of the PC search, we'll also complete the next step of identifying relevant satellite data (`identify_satellite_data`) to apply the same date range to both, and *then* we'll compare results.

In [6]:
def identify_satellite_data_mod(samples: pd.DataFrame,
                           config: FeaturesConfig,
                            candidate_sentinel_meta: pd.DataFrame,
                            sample_item_map: dict
                           ) -> pd.DataFrame:
    """Modified version of 
    cyano.data.utils.satellite_data.identify_satellite_data
    that allows us to change the item map and candidate metadata
    """
    selected_meta = []
    for sample in tqdm(samples.itertuples(), total=len(samples)):
        sample_item_ids = sample_item_map[sample.Index]['sentinel_item_ids']
        if len(sample_item_ids) == 0:
            continue

        sample_items_meta = candidate_sentinel_meta[
            candidate_sentinel_meta.item_id.isin(sample_item_ids)
        ].copy()
        selected_ids = select_items(sample_items_meta, sample.date, config)

        sample_items_meta = sample_items_meta[
            sample_items_meta.item_id.isin(selected_ids)
        ]
        sample_items_meta['sample_id'] = sample.Index

        selected_meta.append(sample_items_meta)

    selected_meta = pd.concat(selected_meta).reset_index(drop=True)
    
    return selected_meta

## Search planetary computer to regenerate results

In [7]:
regenerated_candidate_meta = []
regenerated_map = {}

for sample in tqdm(samples.itertuples(), total=len(samples)):
    # Search planetary computer
    search_results = search_planetary_computer(
        sample.date,
        sample.latitude,
        sample.longitude,
        collections=["sentinel-2-l2a"],
        days_search_window=config.pc_days_search_window,
        meters_search_window=config.pc_meters_search_window,
    )
    
    # Get satellite metadata
    sample_items_meta = get_items_metadata(
        search_results, sample.latitude, sample.longitude, config
    )
    
    regenerated_map[sample.Index]  = {
        "sentinel_item_ids": sample_items_meta.item_id.tolist()
        if len(sample_items_meta) > 0
        else []
    }
    regenerated_candidate_meta.append(sample_items_meta)
    
regenerated_candidate_meta = (
    pd.concat(regenerated_candidate_meta).groupby("item_id", as_index=False)
    .first().reset_index(drop=True)
)
print(f"Generated metadata for {regenerated_candidate_meta.shape[0]:,} Sentinel item candidates")

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.08it/s]

Generated metadata for 325 Sentinel item candidates


In [8]:
regenerated_meta = identify_satellite_data_mod(
    samples = samples,
    config = config,
    candidate_sentinel_meta = regenerated_candidate_meta,
    sample_item_map = regenerated_map
)
regenerated_meta.shape

100%|████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 266.43it/s]


(332, 27)

In [9]:
regenerated_meta.head(2)

,item_id,datetime,platform,min_long,max_long,min_lat,max_lat,eo:cloud_cover,s2:nodata_pixel_percentage,visual_href,...,B07_href,B08_href,B09_href,B11_href,B12_href,B8A_href,SCL_href,WVP_href,days_before_sample,sample_id
0,S2A_MSIL2A_20181028T155341_R054_T18TUM_2020100...,2018-10-28,Sentinel-2A,-77.130866,-76.080027,41.442810,42.447443,99.673942,33.018643,https://sentinel2l2a01.blob.core.windows.net/s...,...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,11,6b94006dfe552540366c57ecfd24663c
1,S2A_MSIL2A_20181031T160411_R097_T18TUM_2020100...,2018-10-31,Sentinel-2A,-77.431180,-76.080020,41.438836,42.447443,40.748203,0.000000,https://sentinel2l2a01.blob.core.windows.net/s...,...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,8,6b94006dfe552540366c57ecfd24663c


## Load results from past PC search

In [10]:
pc_results_dir = (
    AnyPath("s3://drivendata-competition-nasa-cyanobacteria")
    / "data/interim/full_pc_search"
)
loaded_candidate_meta = pd.read_csv(
    pc_results_dir / "sentinel_metadata.csv"
)
assert loaded_candidate_meta.item_id.is_unique
print(
    f"Loaded {loaded_candidate_meta.shape[0]:,} rows of Sentinel candidate metadata"
)

with open(pc_results_dir / "sample_item_map.json", "r") as fp:
    loaded_map = json.load(fp)
print(f"Loaded sample item map for {len(loaded_map):,} samples")

Loaded 56,173 rows of Sentinel candidate metadata
Loaded sample item map for 23,570 samples


In [11]:
loaded_meta = identify_satellite_data_mod(
    samples = samples,
    config=config,
    candidate_sentinel_meta = loaded_candidate_meta,
    sample_item_map = loaded_map
)
loaded_meta.shape

100%|████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 149.75it/s]


(332, 36)

In [12]:
loaded_meta.head(2)

,item_id,datetime,platform,min_long,max_long,min_lat,max_lat,bbox_from_geometry,eo:cloud_cover,s2:water_percentage,...,B07_href,B08_href,B09_href,B11_href,B12_href,B8A_href,SCL_href,WVP_href,days_before_sample,sample_id
0,S2A_MSIL2A_20181028T155341_R054_T18TUM_2020100...,2018-10-28,Sentinel-2A,-77.130866,-76.080027,41.442810,42.447443,False,99.673942,0.006484,...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,11,6b94006dfe552540366c57ecfd24663c
1,S2A_MSIL2A_20181031T160411_R097_T18TUM_2020100...,2018-10-31,Sentinel-2A,-77.431180,-76.080020,41.438836,42.447443,False,40.748203,0.247979,...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,https://sentinel2l2a01.blob.core.windows.net/s...,8,6b94006dfe552540366c57ecfd24663c


## Compare regenerated results with loaded results

In [13]:
loaded_meta.shape, regenerated_meta.shape

((332, 36), (332, 27))

In [14]:
include_cols = ['item_id', 'sample_id', 
                'datetime', 'platform', 'min_long', 'max_long', 'min_lat',
       'max_lat', 'eo:cloud_cover', 's2:nodata_pixel_percentage',
       'days_before_sample']

In [15]:
compare_loaded_meta = (
    loaded_meta.sort_values(by=['sample_id', 'item_id'])[include_cols].round(5)
)
compare_regenerated_meta = (
    regenerated_meta.sort_values(by=['sample_id', 'item_id'])[include_cols].round(5)
)

In [16]:
(compare_loaded_meta == compare_regenerated_meta).all()

item_id                       True
sample_id                     True
datetime                      True
platform                      True
min_long                      True
max_long                      True
min_lat                       True
max_lat                       True
eo:cloud_cover                True
s2:nodata_pixel_percentage    True
days_before_sample            True
dtype: bool